In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import bayesian_pdes as bpdes
from bayesian_pdes.util import plotting
plotting.monkeypatch_animation()
import numpy as np
import sympy as sp

In [2]:
Tend = 1.
L = 1.

In [3]:
def ic_fun(x):
    #return np.zeros((x.shape[0], 1))
    return 1./(1+np.exp(-20*(x-0.5)))

In [4]:
x, y, ls_x = sp.symbols('x, y, ls_x')
def A(k):
    return k.diff(x,x)
def A_bar(k):
    return k.diff(y,y)
def B(k):
    return k.diff(x)
def B_bar(k):
    return k.diff(y)

k = sp.exp(-(x-y)**2 / (2.*ls_x**2))
op_system = bpdes.operator_compilation.sympy_gram.compile_sympy([A, B], [A_bar, B_bar], k, [[x], [y], [ls_x]])

In [18]:
dt = 0.05
dx = 0.1
theta = 1.0
target_dt = 0.01
skip = max(int(target_dt/dt), 1)

x_pts = np.arange(0,L+dx,dx)[:,None]
x_int = x_pts[1:-1,:]
x_bdy = x_pts[[0,-1], :]
times = np.arange(0,Tend+dt,dt)
def time_design_function(t):
    return x_pts
def other_obs_function(t):
    bdy_term = np.zeros((x_bdy.shape[0], 1))
    return [(x_bdy, bdy_term)]

fun_args = np.array([2*dx])

In [19]:
ic = [(x_pts, ic_fun(x_pts))]

In [20]:
posteriors = bpdes.parabolic.solve_theta(
    [A, A_bar], 
    [[B], [B_bar]], 
    op_system, 
    time_design_function, 
    other_obs_function,
    ic,
    times,
    theta,
    fun_args
)

In [21]:
# deterministic solution by finite difference
import scipy.sparse, scipy.sparse.linalg
dt_det = 1e-4
alpha_det = 1.0
theta_det = 0.5
dx_det = 0.01

points_det = np.arange(0, L+dx_det, dx_det)

times_det = np.arange(0, Tend+dt_det, dt_det)
initial = ic_fun(points_det)
us = [initial]
u_curr = initial

ones = np.ones(points_det.shape[0])
main_mat = scipy.sparse.diags([-(1-theta_det)*alpha_det*ones[:-1]*dt_det/dx_det**2, 
                               1+2*(1-theta_det)*alpha_det*ones*dt_det/dx_det**2, 
                               -(1-theta_det)*alpha_det*ones[:-1]*dt_det/dx_det**2
                              ], [-1,0,1], format='csr')
rhs_mat = scipy.sparse.diags([theta_det*alpha_det*ones[:-1]*dt_det/dx_det**2, 
                               1 - 2*theta_det*alpha_det*ones*dt_det/dx_det**2, 
                               theta_det*alpha_det*ones[:-1]*dt_det/dx_det**2
                              ], [-1,0,1], format='csr')
main_mat[0,0] = -1/dx_det
main_mat[0,1] = 1/dx_det
main_mat[-1,-1] = 1/dx_det
main_mat[-1,-2] = -1/dx_det
rhs_mat[0,0] = 0
rhs_mat[0,1] = 0
rhs_mat[-1, -1] = 0
rhs_mat[-1, -2] = 0
for t in times_det[1:]:
    rhs = rhs_mat.dot(u_curr)
    u_curr = scipy.sparse.linalg.spsolve(main_mat, rhs)
    us.append(u_curr)
det_skip = int(dt / dt_det)

In [22]:
save_at = 'insulated_theta={}_dx={}_dt={}.mp4'.format(theta, dx, dt)
plt.xlim(0, L)
plt.ylim(0,1)
a = plotting.plot_parabolic_1d(x_pts, posteriors, times, stride=skip, deterministic=(points_det, us[::det_skip]))
a.save(save_at, fps=20)
a